# PDF to Text

## Import Package

In [ ]:
# # Visual Python: Data Analysis > PDF
# !pip install PyMuPDF

# # Visual Python: Data Analysis > PDF
# !pip install nltk

# # Visual Python: Logic > code
# import pandas as pd
# import fitz
# import nltk
# nltk.download('punkt')

# # Visual Python: Logic > code
# def vp_pdf_get_sentence(fname_lst):
#     '''
#     Get sentence from pdf file by PyMuPDF
#     '''
#     df = pd.DataFrame()
#     for fname in fname_lst:
#         if fname.split('.')[-1] != 'pdf': continue
#         try:
#             doc = fitz.open(fname)
#             sentence_lst = []
#             for page in doc:
#                 block_lst = page.get_text('blocks')

#                 text_lst = [block[4] for block in block_lst if block[6] == 0]
#                 text = '\n'.join(text_lst)

#                 sentence_lst.extend([sentence for sentence in nltk.sent_tokenize(text)])

#             doc.close()
#         except Exception as e:
#             print(e)
#             continue

#         df_doc = pd.DataFrame({
#             'fname': fname.split('/')[-1],
#             'sentence': sentence_lst
#         })
#         df = pd.concat([df,df_doc])

#     return df.reset_index().drop('index', axis=1)

## Transformation

In [ ]:
# Visual Python: Data Analysis > PDF
pdf_lst = ['./drive/MyDrive/Colab Notebooks/ESG/Carrefour 2022.pdf']
carrefour_2022 = vp_pdf_get_sentence(pdf_lst)

pdf_lst = ['./drive/MyDrive/Colab Notebooks/ESG/Carrefour 2023.pdf']
carrefour_2023 = vp_pdf_get_sentence(pdf_lst)

pdf_lst = ['./drive/MyDrive/Colab Notebooks/ESG/Coca cola 2022.pdf']
cocacola_2022 = vp_pdf_get_sentence(pdf_lst)

pdf_lst = ['./drive/MyDrive/Colab Notebooks/ESG/Coca cola 2023.pdf']
cocacola_2023 = vp_pdf_get_sentence(pdf_lst)

## Preprocessing

In [ ]:
# 불필요 열 제거

carrefour_2022.drop(columns = ["fname"], inplace = True)
carrefour_2023.drop(columns = ["fname"], inplace = True)

cocacola_2022.drop(columns = ["fname"], inplace = True)
cocacola_2023.drop(columns = ["fname"], inplace = True)

In [ ]:
# 기업별 파일 병합

carrefour_report = pd.concat([carrefour_2022,carrefour_2023], join='outer', axis=0, ignore_index=True)
cocacola_report = pd.concat([cocacola_2022,cocacola_2023], join='outer', axis=0, ignore_index=True)



# 열 이름 변경

carrefour_report.rename(columns = {"sentence": "Sentence"}, inplace = True)
cocacola_report.rename(columns = {"sentence": "Sentence"}, inplace = True)

In [ ]:
# 키보드 상에 존재하지 않는 문자 제거
carrefour_report["Preprocessed"] = carrefour_report["Sentence"].str.replace(r'[^!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\0-9a-zA-Z]', ' ', regex=True)
cocacola_report["Preprocessed"] = cocacola_report["Sentence"].str.replace(r'[^!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\0-9a-zA-Z]', ' ', regex=True)
# 소문자로 변환
carrefour_report["Preprocessed"] = carrefour_report["Preprocessed"].str.lower()
cocacola_report["Preprocessed"] = cocacola_report["Preprocessed"].str.lower()

In [ ]:
import re
from nltk.corpus import stopwords

# nltk의 불용어 리스트를 다운로드
import nltk
nltk.download('stopwords')

# 불용어 리스트 (영어로 설정)
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_comment(comment):
    # 입력이 문자열이 아닌 경우 빈 문자열 반환
    if not isinstance(comment, str):
        return ""

    # 7) 접속사, 대명사, 조사, 비동사 등 불용어 삭제
    tokens = comment.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # 4) 숫자 삭제
    comment = re.sub(r'\d+', '', comment)

    # 리스트를 공백으로 연결하여 하나의 문자열로 반환
    return ' '.join(filtered_tokens)

In [ ]:
carrefour_report['Preprocessed'] = carrefour_report['Preprocessed'].apply(clean_comment)
cocacola_report['Preprocessed'] = cocacola_report['Preprocessed'].apply(clean_comment)

### Stemming

자연어 처리 과정에서 **Stemming**과 **Lemmatizing**은 모두 단어의 기본 형태를 찾기 위해 사용하는 기법입니다. 하지만 이 둘은 약간 다른 방식으로 작동하며, 프로젝트의 목적과 데이터의 특성에 따라 둘 중 하나를 선택하거나, 때로는 둘 다 사용할 수 있습니다.

**Stemming**
- **정의:** Stemming은 단어의 어간(단어의 변하지 않는 기본 형태)을 찾기 위해 단어의 접미사나 어미를 단순히 잘라내는 작업입니다. 예를 들어, "running"은 "run"으로, "better"는 "better"로 변환됩니다.
- **장점:** 매우 빠르고 간단합니다.
- **단점:** 단순히 접미사를 자르기 때문에, 의미가 조금 왜곡될 수 있습니다. 예를 들어 "running"과 "runner"가 모두 "run"으로 변환될 수 있습니다.  
- **TIP: 이후 진행할 키워드 유사도 분석과 연관지어 Stemming만 사용해도 괜찮을 명확한 이유를 서술해야 한다.**

**Lemmatizing**
- **정의:** Lemmatizing은 단어의 기본 사전형(lemma)을 찾는 작업입니다. 이를 위해 단어의 품사와 의미를 고려하여 정확한 형태로 변환합니다. 예를 들어, "better"는 "good"으로, "running"은 "run"으로 변환됩니다.
- **장점:** 보다 의미에 맞는 변환을 제공하며, Stemming보다 정교합니다.
- **단점:** 처리 속도가 느리고, 복잡합니다.

**둘 다 사용하는 경우**
- 일반적으로 둘 다 사용하는 경우는 많지 않습니다. 대부분의 프로젝트에서는 **Lemmatizing**이 더 의미론적으로 정확한 결과를 제공하기 때문에 선호됩니다.
- **Stemming**은 속도가 중요하거나 단어의 세부적인 의미가 덜 중요할 때 사용됩니다.
- 일부 경우에는 Stemming으로 데이터의 크기를 줄인 후, Lemmatizing을 적용해 더 정교한 처리를 하는 방식이 사용될 수도 있습니다. 그러나 이는 특정한 상황에 한정됩니다.

**결론**  
보편적으로는 **둘 중 하나**를 선택해 사용합니다. **Lemmatizing**이 의미론적으로 더 정확하기 때문에 더 많이 사용되지만, 프로젝트의 목표와 필요에 따라 **Stemming**을 선택하는 것도 충분히 유효한 접근입니다.

[PorterStemmer 참고자료](https://towardsai.net/p/l/stemming-porter-vs-snowball-vs-lancaster)

In [ ]:
import re
from nltk.stem import SnowballStemmer    # PorterStemmer의 개선판

stemmer = SnowballStemmer("english")



# 전처리 함수 정의

def preprocess_word(word):
  word = re.sub(r'(.)\1{2,}', r'\1\1', word)    # 동일 알파벳 세번 연속 등장 시 두 개만 남기기
  return word



# Stemming 함수 정의

def f_stem(text):
  if isinstance(text, str):
    words = text.split()
    stemmed_words = []

    for word in words:    # 단어가 #으로 시작하면 무시(삭제)
      if word.startswith("#"):
        continue

      preprocessed_word = preprocess_word(word)
      stemmed_word = stemmer.stem(preprocessed_word)

      stemmed_words.append(stemmed_word)

    return ' '.join(stemmed_words)
  else:
    return text    # 문자열이 아니면 그대로 반환

In [ ]:
carrefour_report["Preprocessed"] = carrefour_report["Preprocessed"].apply(f_stem)
cocacola_report["Preprocessed"] = cocacola_report["Preprocessed"].apply(f_stem)

# Export

In [ ]:
carrefour_report

,Sentence,Preprocessed
0,2\n\nUNIVERSAL REGISTRATION DOCUMENT 2022 / ...,2 univers registr document 2022 / carrefour 49...
1,2\n\nwww.carrefour.com\nUNIVERSAL REGISTRATION...,2 ww.carrefour.com univers registr document 20...
2,Chapter 1 presents Carrefour’s raison d’être ...,chapter 1 present carrefour raison tre ambit b...
3,"In line with this \nambition, this chapter als...","line ambition, chapter also look project devel..."
4,"Lastly, it reviews the \nGroup’s CSR performan...","lastly, review group csr perform summari achie..."
...,...,...
3409,In line \nwith the commitments made at Group l...,"line commit made group level, charter requir f..."
3410,They \nensure that these principles are applie...,"ensur principl applied, organis check observ v..."
3411,"Lastly, they authorise the \nCarrefour group o...","lastly, authoris carrefour group authoris pers..."
3412,"not engage in slavery, debt bondage or forced ...","engag slavery, debt bondag forc compulsori lab..."


In [ ]:
cocacola_report

,Sentence,Preprocessed
0,Refresh \nthe World.,refresh world.
1,Make a \nDifference.,make difference.
2,2022 BUSINESS & SUSTAINABILITY REPORT,2022 busi & sustain report
3,2\nTHE COCA-COLA COMPANY 2022 BUSINESS & SUSTA...,2 coca-cola compani 2022 busi & sustain report...
4,Our growth \nstrategy is grounded in our core ...,growth strategi ground core valu commit social...
...,...,...
1579,5\t The reported priority agricultural ingred...,5 report prioriti agricultur ingredi volum pro...
1580,The exclusions are related to \n\nour innocent...,"exclus relat innocent, costa adan busi follow ..."
1581,Click here to access Consolidated Business exc...,click access consolid busi exclus relat agricu...
1582,11\nCONTENTS\nWATER\nABOUT THIS UPDATE\nPACKAG...,11 content water updat packag climat agricultu...


In [ ]:
# Visual Python: Data Analysis > File
carrefour_report.to_csv('./drive/MyDrive/Colab Notebooks/ESG/carrefour_report_preprocessed.csv', encoding='cp949')

In [ ]:
# Visual Python: Data Analysis > File
cocacola_report.to_csv('./drive/MyDrive/Colab Notebooks/ESG/cocacola_report_preprocessed.csv', encoding='cp949')